In [1]:
#%%
import numpy as np
import pdb

In [2]:
#%% Initialisations
class Agent():
    def __init__(self):
        self.row = 4
        self.column = 4

        # States in the grid world
        self.state1 = np.empty((self.row,self.column))

        # Value for each state
        self.state_value = {}
        value = 1
        for i in range(self.row):
            for j in range(self.column):
                self.state1[i,j] = value
                self.state_value[value] = 0
                value +=1

        # Action space
        self.action = {6: 'left', 7: 'right', 8: 'north', 9: 'south'}
        
        # Porbability of picking each action.
#        pi_act
        
        # Action space (numerical).
        act_n = np.arange(6,10)
        
        # Policy mapping. 
        self.act_s = {}
        for i in range(self.row):
            for j in range(self.column):
                self.act_s[self.state1[i,j]] = np.random.choice(act_n)

    # State transition and reward.
    def action_state(self,action,state):
        reward = 0
        state1 = np.inf
        loop = True
        #leftmost corner condition
        if state in (self.state1[3,0],self.state1[1,0],self.state1[2,0]):
            if action == self.action[6]:  #left
                reward = -1
                state1 = state
            if action == self.action[7]:  #right
                reward = -1
                state1 = state + 1
            if action == self.action[8]: #north
                state1 = state - 4
                if state1 == self.state1[0,0]:
                    reward = 0
                    state1 = 1
                else:
                    reward  = -1
                    state1 = state - 4
            if action == self.action[9]: #south
                state1 = state + 4
                if state1 not in self.state1:
                    reward = -1
                    state1 = state
                else:
                    reward = -1
                    state1 = state + 4
            loop = False

        #right most condition
        if state == self.state1[0,3] or state == self.state1[1,3] or state == self.state1[2,3]:
            if action == self.action[6]: #left
                reward = -1
                state1 = state - 1
            if action == self.action[7]: #right
                reward = -1
                state1 = state
            if action == self.action[8]: # north
                state1 = state - 4
                if state1 not in self.state1:
                    reward = -1
                    state1 = state
                else:
                    reward = -1
                    state1 = state - 4
            if action == self.action[9]: #south
                state1 = state + 4
                if state1 == self.state1[3,3]:
                    reward = 0
                    state1 = 16
                else:
                    reward = -1
                    state1 = state + 4
            loop = False


        elif loop == True:
            if action == self.action[6]: #left
                state1 = state - 1
                if state1 ==  self.state1[0,0]:
                    reward = 0
                    state1 = 1
                else:
                    reward = -1
                    state1 = state - 1

            if action == self.action[7]: #right
                state1 = state + 1
                if state1 == self.state1[3,3]:
                    reward = 0
                    state1 = 16
                else:
                    reward = -1
                    state1 = state + 1

            if action == self.action[8]: #north
                state1 = state - 4
                if state1 not in self.state1:
                    reward = -1
                    state1 = state
                else:
                    reward = -1
                    state1 = state - 4

            if action == self.action[9]: #south
                state1 = state + 4
                if state1 not in self.state1:
                    reward = -1
                    state1 = state
                else:
                    reward = -1
                    state1 = state + 4
        return state1, reward


  #choosing any action with equal probability
    def policy(self,state):
        ac = np.arange(6,10)
        action2 = []
        for j in range(len(ac)):
            action1 = self.action[ac[j]]
            action2.append(action1)
        action = np.random.choice(list(self.action.keys()))
        return action           

In [3]:
#%% Policy iteration
# Initialize the agent.
a = Agent()

# Total number of states
num_states = a.row*a.column

# Total number of actions
action_space = a.action.keys()
num_actions = len(action_space)

# Numerical action space.
act_space = np.arange(6,10)

# Discount factor
gamma = 0.99

# Convergence variable
theta = 0.0001

# Counter just to keep track
count = 0

# Loop termination condition
loop1  = True

#temp values for each state
ac_values = np.zeros((num_states,num_actions))

# Policy loop
policy_stable = False

# Policy counter
plc_cnt = 0

# Optimal policy
pol = ["" for x in range(num_states)]
action3 = ["" for x in range(num_actions)]

#############
# Algorithm
#############

while policy_stable == False:
    #pdb.set_trace()
    
    
    while loop1 == True:
        #policy evaluation
        # Initialize delta to 0.
        delta = 0.0
        
        # Value convergence loop.
        val_con = 0
        # Loop over 2 - 15 (1 and 16 are terminal states).
        for i in range(2,num_states):
            # State value of a particular state
            v = np.copy(a.state_value[i])
    
            # Action for the state i.
            cur_act = a.action[a.act_s[i]]

            [state_transit,rew] = a.action_state(cur_act, i)

            # Discounted updates
            temp = 1.0*(rew + gamma*np.copy(a.state_value[state_transit]))
            
            a.state_value[i] = np.copy(temp)
        
            # v-V(s), compare for a particular state
            final = temp - v
            delta = max(delta, abs(final))

            # Policy evaluation convergence check.
            if delta<theta:
                val_con = val_con + 1
                loop1 = False
        
#        if val_con == 14:
#            #pdb.set_trace()
#            loop1 = False
       
    # Policy convergence check.
    pol_con = 0
    # Policy improvement starts
    for i in range(2,num_states):
        new_action = []
        # Old action.
        old_action = a.act_s[i]
        
        # State action pair.
        temp_s_a = np.zeros((len(act_space),1))
        
        for j in range(len(act_space)):
            [state_transit,rew] = a.action_state(a.action[act_space[j]], i)
            temp_s_a[j,0] = rew + gamma*a.state_value[state_transit]
        
        # New policy.
        #pdb.set_trace()
        pi_cur = np.argmax(temp_s_a) + 6
        
        if old_action != pi_cur:
            a.act_s[i] = pi_cur
        else:
            pol_con = pol_con+1
            
    if pol_con == num_states-2:
        policy_stable = True
        loop1 = False
         
                
    count = count +1
    print(count)

1
2


In [4]:
#%% Plot the policy
        
def plot_pol(act):
    if act == 6:
        temp = '|' + chr(8592) + '|'
    elif act == 7:
        temp = '|' +  chr(8594) + '|'
    elif act == 8:
        temp = '|' + chr(8593) + '|'
    elif act == 9:
        temp = '|' + chr(8595) + '|'
        
    return temp

plt_pol = []
plt_pol.append('| |')
cnt = 0
cnt_st = 0
for i in range(2,num_states):
    if np.mod(i,4)==0:
        plt_pol[cnt] = plt_pol[cnt] + plot_pol(a.act_s[i])
        cnt = cnt + 1
        plt_pol.append('')
    else:
        plt_pol[cnt] = plt_pol[cnt] + plot_pol(a.act_s[i])
plt_pol[cnt] = plt_pol[cnt] + ('| |')


for i in range(4):
    print(plt_pol[i])

| ||←||←||←|
|↑||←||←||←|
|↑||←||↓||↓|
|←||→||→|| |
